In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import numpy as np
import pandas as pd
import collections

In [ ]:
x_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터3] samp_cst_feat'}.csv")
df_x = pd.read_csv(x_name, index_col=0)

y_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터2] samp_train'}.csv")
df_y = pd.read_csv(y_name, index_col=0)

df = pd.merge(df_x, df_y, on='cst_id_di')

In [ ]:
df.MRC_ID_DI[df.MRC_ID_DI > 0 ] = 1

In [ ]:
nc_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터4] variable_dtype'}.xlsx")
nc = pd.read_excel(nc_name, index_col=0)

In [ ]:
df = df.drop(columns = ['MRC_ID_DI', 'VAR021', 'VAR046'], axis=1)

In [ ]:
df[df.columns] = (df[df.columns]+1)/2*255

In [ ]:
from PIL import Image
import numpy as np
for ind, k in enumerate(df.index):
    data = np.zeros((224, 224, 3), dtype=np.uint8)
    for i in range(224):
        for j in range(224):
            data[i, j] = [df.iloc[ind, j], df.iloc[ind, j], df.iloc[ind, j]]
    img = Image.fromarray(data, 'RGB')
    img_name = os.path.join(os.getcwd(), 'image', str(k) + '.png')
    img.save(img_name)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import math
from glob import glob
import os

master_path = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터2] samp_train'}.csv")
master = pd.read_csv(master_path)
master.MRC_ID_DI[master.MRC_ID_DI > 0 ] = 1

In [ ]:
master_0 = master[master['MRC_ID_DI'] == 0].sample(frac=1)
master_1 = master[master['MRC_ID_DI'] == 1].sample(frac=1)

sample_size = len(master_0) if len(master_0) < len(master_1) else len(master_1)

master = pd.concat([master_0.head(sample_size), master_1.head(sample_size)]).sample(frac=1)

master_copy = master.copy()
train_set = master_copy.sample(frac=0.8, random_state=42)
test_set = master_copy.drop(train_set.index)

In [ ]:
train_set

In [ ]:
img_path = os.path.join(os.getcwd(), 'image')

y_train = []
X_train_paths = []

y_test = []
X_test_paths = []

for i in range(len(master)):
    if master.iloc[i, 0] in train_set.cst_id_di.values:
        X_train_paths.append( os.path.join(img_path, str(master.iloc[i, 0]) +'.png') )
        y_train.append(master.iloc[i, 1])
    else:
        X_test_paths.append( os.path.join(img_path, str(master.iloc[i, 0]) +'.png') )
        y_test.append(master.iloc[i, 1])

y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
X_train_paths

In [ ]:
X_train = []
for file_path in X_train_paths:
    #read image
    img = cv2.imread(file_path)
    X_train.append(img)

X_train = np.array(X_train)

X_test = []
for file_path in X_test_paths:
    #read image
    img = cv2.imread(file_path)
    X_test.append(img)

X_test = np.array(X_test)

In [ ]:
import tensorflow as tf
y_train = tf.keras.utils.to_categorical(y_train)

In [ ]:
val_split_num = int(round(0.2*len(y_train)))
x_t = X_train[val_split_num:]
y_t = y_train[val_split_num:]
x_v = X_train[:val_split_num]
y_v = y_train[:val_split_num]

print('x_train', x_t.shape)
print('y_train', y_t.shape)
print('x_test', x_v.shape)
print('y_test', y_v.shape)

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [ ]:
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense

img_rows, img_cols, img_channel = 224, 224, 3

base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

In [ ]:
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(units=2, activation="softmax"))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

In [ ]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                      exception_only=False, running_compiled_code=False):
       etype, value, tb = sys.exc_info()
       return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

batch_size = 32
epochs = 50

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(x_t)


history = model.fit_generator(
    train_datagen.flow(x_t, y_t, batch_size=batch_size),
    steps_per_epoch=x_t.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(x_v, y_v),
    callbacks=[ModelCheckpoint('VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]
)

In [ ]:
a